In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input


In [2]:
# (Dans l'autre notebook)
import pandas as pd
df_for_ml = pd.read_csv("./df_for_ml.csv")

- Séparation en train - Cross Validation - Test set

In [3]:
feature_cols = [c for c in df_for_ml.columns if c.startswith("f_")]
X = df_for_ml[feature_cols].to_numpy(dtype=np.float32)
y = df_for_ml["emotion_label"].to_numpy()

# 75% train, 15% val, 10% test
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_tmp, y_tmp, test_size=0.15, random_state=42, stratify=y_tmp
)



le = LabelEncoder()
le.fit(y_train)                      # fit sur TRAIN uniquement
y_train_int = le.transform(y_train)
y_val_int   = le.transform(y_val)
y_test_int  = le.transform(y_test)


print(X_train.shape, X_val.shape, X_test.shape)


(1101, 120) (195, 120) (144, 120)


- Centrer réduire les données d'entrainement

In [ ]:


# créer l'objet de centrage réduction 
scaler = StandardScaler()                

# Apprendre les paramètres (moyenne/écart-type) sur le TRAIN uniquement
X_train = scaler.fit_transform(X_train).astype(np.float32)

# Réutiliser le MÊME scaler pour centrer/réduire VAL et TEST (pas de fit ici)
X_val   = scaler.transform(X_val).astype(np.float32)
X_test  = scaler.transform(X_test).astype(np.float32)



# Conception des modèles

In [29]:
#Hyperparamètres
input_dim = X_train.shape[1]
num_classes = len(np.unique(y))
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 50


In [55]:
import tensorflow as tf

def make_mse_onehot(num_classes: int):
    @tf.function
    def mse_onehot(y_true, y_pred):
        # y_true peut être (batch,) ou (batch,1) -> on squeeze et on cast
        y_true = tf.cast(tf.squeeze(y_true), tf.int32)
        y_true_oh = tf.one_hot(y_true, depth=num_classes)
        return tf.reduce_mean(tf.math.squared_difference(y_true_oh, y_pred))
    return mse_onehot


In [57]:
def build_model_A(input_dim: int, num_classes: int) -> keras.Model:
    """Modèle A — peu profond"""
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(32, activation="relu")(inputs)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="Model_A")
    mse_onehot = make_mse_onehot(num_classes)
    model.compile(
        optimizer=keras.optimizers.Adam(LEARNING_RATE),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[mse_onehot, "accuracy"],
    )
    return model

def build_model_B(input_dim: int, num_classes: int) -> keras.Model:
    """Modèle B"""
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation="relu")(inputs)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="Model_B")
    mse_onehot = make_mse_onehot(num_classes)
    model.compile(
        optimizer=keras.optimizers.Adam(LEARNING_RATE),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[mse_onehot, "accuracy"],
    )
    return model

def build_model_C(input_dim: int, num_classes: int) -> keras.Model:
    """Modèle C"""
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(128, activation="relu")(inputs)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="Model_C")
    mse_onehot = make_mse_onehot(num_classes)
    model.compile(
        optimizer=keras.optimizers.Adam(LEARNING_RATE),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[mse_onehot, "accuracy"],
    )
    return model

def build_model_D(input_dim: int, num_classes: int) -> keras.Model:
    """Modèle D — le plus profond"""
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(256, activation="relu")(inputs)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="Model_D")
    mse_onehot = make_mse_onehot(num_classes)
    model.compile(
        optimizer=keras.optimizers.Adam(LEARNING_RATE),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[mse_onehot, "accuracy"],
    )
    return model

# Exemple d’instanciation :
# model_A = build_model_A(input_dim, num_classes)
# model_B = build_model_B(input_dim, num_classes)
# model_C = build_model_C(input_dim, num_classes)
# model_D = build_model_D(input_dim, num_classes)


# --- 2) Callbacks communs (optionnels mais conseillés) ---
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5),
]



In [58]:

# ==== Instanciation ====
model_A = build_model_A(input_dim, num_classes)
model_B = build_model_B(input_dim, num_classes)
model_C = build_model_C(input_dim, num_classes)
model_D = build_model_D(input_dim, num_classes)


In [59]:
def mse_onehot_numpy(y_true_int, y_pred_proba, num_classes):
    Y = np.eye(num_classes, dtype=np.float32)[y_true_int]   # (N, C)
    return np.mean((Y - y_pred_proba) ** 2)

In [60]:

# ==== Entraînement ====
histories = {}


print("\n=== Entraînement model_A ===")
histories["A"] = model_A.fit(
    X_train, y_train_int, validation_data=(X_val, y_val_int),
    epochs=EPOCHS, batch_size=BATCH_SIZE,  verbose=1
)

print("\n=== Entraînement model_B ===")
histories["B"] = model_B.fit(
    X_train, y_train_int, validation_data=(X_val, y_val_int),
    epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1
)


print("\n=== Entraînement model_C ===")
histories["C"] = model_C.fit(
    X_train, y_train_int, validation_data=(X_val, y_val_int),
    epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1
)

print("\n=== Entraînement model_D ===")
histories["D"] = model_D.fit(
    X_train, y_train_int, validation_data=(X_val, y_val_int),
    epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1
)




=== Entraînement model_A ===
Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1717 - loss: 2.4372 - mse_onehot: 0.1175 - val_accuracy: 0.1795 - val_loss: 2.2732 - val_mse_onehot: 0.1167
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2498 - loss: 2.0402 - mse_onehot: 0.1078 - val_accuracy: 0.2256 - val_loss: 2.0561 - val_mse_onehot: 0.1097
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3052 - loss: 1.8418 - mse_onehot: 0.0992 - val_accuracy: 0.2974 - val_loss: 1.9276 - val_mse_onehot: 0.1053
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3524 - loss: 1.7086 - mse_onehot: 0.0962 - val_accuracy: 0.2923 - val_loss: 1.8449 - val_mse_onehot: 0.1018
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4124 - loss: 1.6061 - mse_onehot: 0.0922 - val_accuracy: 0.3231 - val_loss: 1.7988 - val_mse_onehot: 0.0998
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4559 - loss: 1.5300 - mse_onehot: 0.087

- Evaluation du modèle

In [61]:


models = [
    ("A (peu profond)",  model_A),
    ("B",                model_B),
    ("C",                model_C),
    ("D (plus profond)", model_D),
]

results = []
for name, model in models:
    p_tr = model.predict(X_train, verbose=0)
    p_va = model.predict(X_val,   verbose=0)

    mse_train = mse_onehot_numpy(y_train_int, p_tr, num_classes)
    mse_val   = mse_onehot_numpy(y_val_int,   p_va, num_classes)

    acc_train = (p_tr.argmax(axis=1) == y_train_int).mean()
    acc_val   = (p_va.argmax(axis=1) == y_val_int).mean()

    results.append({
        "model": name,
        "mse_train": mse_train,
        "mse_val": mse_val,
        "acc_train": acc_train,
        "acc_val": acc_val
    })

summary_df = pd.DataFrame(results)
print(summary_df)


              model     mse_train   mse_val  acc_train   acc_val
0   A (peu profond)  1.477657e-02  0.087207   0.949137  0.528205
1                 B  1.200145e-04  0.087142   1.000000  0.574359
2                 C  8.723692e-06  0.079032   1.000000  0.610256
3  D (plus profond)  7.718055e-08  0.074291   1.000000  0.635897


In [111]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers

def build_model_A_bis_dropout(input_dim: int, num_classes: int,
                              dropout_rate: float = 0.3,
                              l2_weight: float = 0.0) -> keras.Model:
    """Modèle A — deux couches + Dropout (et option L2)"""

    inputs = layers.Input(shape=(input_dim,))

    # 1ʳᵉ couche cachée
    x = layers.Dense(
        40, activation="relu",
        kernel_regularizer=regularizers.l2(l2_weight) if l2_weight > 0 else None
    )(inputs)

    x = layers.Dropout(dropout_rate)(x)

    # 2ᵉ couche cachée
    x = layers.Dense(
        20, activation="relu",
        kernel_regularizer=regularizers.l2(l2_weight) if l2_weight > 0 else None
    )(x)

    x = layers.Dropout(dropout_rate)(x)

    # 3 eme couche
    x = layers.Dense(
        10, activation="relu",
        kernel_regularizer=regularizers.l2(l2_weight) if l2_weight > 0 else None
    )(x)

    # Dropout après la 2ᵉ couche cachée
    # x = layers.Dropout(dropout_rate)(x)
    # x = Dropout(0.25)(x)   

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="Model_A_bis_Dropout_2layers")

    mse_onehot = make_mse_onehot(num_classes)
    model.compile(
        optimizer=keras.optimizers.Adam(LEARNING_RATE),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[mse_onehot, "accuracy"],
    )
    return model

# Utilisation
model_A_bis = build_model_A_bis_dropout(input_dim, num_classes, l2_weight=0.01, dropout_rate=0.2)

print("\n=== Entraînement model_A_bis (2 couches + Dropout) ===")
histories["A_bis_do"] = model_A_bis.fit(
    X_train, y_train_int,
    validation_data=(X_val, y_val_int),
    epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1
)



=== Entraînement model_A_bis (2 couches + Dropout) ===
Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1653 - loss: 3.1341 - mse_onehot: 0.1126 - val_accuracy: 0.1846 - val_loss: 2.9777 - val_mse_onehot: 0.1076
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1871 - loss: 2.9863 - mse_onehot: 0.1090 - val_accuracy: 0.2154 - val_loss: 2.9040 - val_mse_onehot: 0.1072
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1962 - loss: 2.8925 - mse_onehot: 0.1075 - val_accuracy: 0.2205 - val_loss: 2.8280 - val_mse_onehot: 0.1065
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2234 - loss: 2.8077 - mse_onehot: 0.1063 - val_accuracy: 0.2205 - val_loss: 2.7536 - val_mse_onehot: 0.1057
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2352 - loss: 2.7304 - mse_onehot: 0.1052 - val_accuracy: 0.2359 - val_loss: 2.6766 - val_mse_onehot: 0.1047
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2598 - loss: 2

In [113]:


def evaluate_model(model, name, 
                   X_train, y_train_int, 
                   X_val,   y_val_int, 
                   num_classes):
    # Prédictions
    p_tr = model.predict(X_train, verbose=0)
    p_va = model.predict(X_val,   verbose=0)

    # MSE (one-hot) et accuracies
    mse_train = mse_onehot_numpy(y_train_int, p_tr, num_classes)
    mse_val   = mse_onehot_numpy(y_val_int,   p_va, num_classes)

    acc_train = (p_tr.argmax(axis=1) == y_train_int).mean()
    acc_val   = (p_va.argmax(axis=1) == y_val_int).mean()

    # DataFrame identique au tableau précédent
    return pd.DataFrame([{
        "model": name,
        "mse_train": mse_train,
        "mse_val": mse_val,
        "acc_train": acc_train,
        "acc_val": acc_val
    }])

# --- Exemple d’utilisation ---
# Pour le modèle C :
summary_df = evaluate_model(model_A_bis, "A_bis", 
                            X_train, y_train_int, 
                            X_val,   y_val_int, 
                            num_classes)
print(summary_df)


   model  mse_train   mse_val  acc_train   acc_val
0  A_bis   0.035001  0.072166   0.823797  0.538462
